In [27]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

df = pd.read_csv('..\data\df_merged.csv',index_col=0)
df = df.fillna(method='ffill')
df = df.set_index('日期')
df = df.astype('float64')
display(df.head())
display(df.tail())

,H-110 COT,H-120 COT,H-130 COT,H-140 COT,H-150 COT,H-160 COT,H-170 COT,H-180 COT,H-190 COT,n-P值,i-P值,n+i P值,Aromatics,N,O,乙烯產量,丙烯產量
日期,,,,,,,,,,,,,,,,,
2018-08-01 00:00:00,820.046387,29.800571,820.729370,820.600525,820.039612,819.586975,820.273438,821.509033,820.206848,42.89,39.77,82.66,3.24,13.31,0.48,2125.68,944.59
2018-08-01 00:10:00,818.146851,29.580025,818.534668,818.692932,818.604431,816.963257,818.849060,816.913757,819.320862,42.89,39.77,82.66,3.24,13.31,0.48,2125.68,944.59
2018-08-01 00:20:00,818.323975,29.494396,819.611572,819.289856,820.808228,819.384216,819.460083,820.118835,819.244202,42.89,39.77,82.66,3.24,13.31,0.48,2125.68,944.59
2018-08-01 00:30:00,819.604675,29.330263,819.857727,819.296204,819.349243,819.066467,818.818909,818.713257,819.158691,42.89,39.77,82.66,3.24,13.31,0.48,2125.68,944.59
2018-08-01 00:40:00,821.526184,29.456963,819.666077,820.332581,820.627075,820.719360,819.730835,821.595703,820.747253,42.89,39.77,82.66,3.24,13.31,0.48,2125.68,944.59


,H-110 COT,H-120 COT,H-130 COT,H-140 COT,H-150 COT,H-160 COT,H-170 COT,H-180 COT,H-190 COT,n-P值,i-P值,n+i P值,Aromatics,N,O,乙烯產量,丙烯產量
日期,,,,,,,,,,,,,,,,,
2021-06-30 23:10:00,27.408436,27.526560,27.766571,27.404127,26.939030,27.202906,26.899256,26.655487,27.490553,37.91,38.61,76.52,3.93,17.36,1.11,572.235,316.125
2021-06-30 23:20:00,27.407352,27.488029,27.798372,27.398758,26.901344,27.243582,26.871965,26.637991,27.498505,37.91,38.61,76.52,3.93,17.36,1.11,572.235,316.125
2021-06-30 23:30:00,27.393206,27.499577,27.752710,27.399584,26.891518,27.195532,26.831932,26.626566,27.481943,37.91,38.61,76.52,3.93,17.36,1.11,572.235,316.125
2021-06-30 23:40:00,27.396896,27.454544,27.733051,27.381708,26.855377,27.172764,26.830599,26.643572,27.473854,37.91,38.61,76.52,3.93,17.36,1.11,572.235,316.125
2021-06-30 23:50:00,27.360914,27.440886,27.734583,27.331905,26.831297,27.138718,26.834709,26.599140,27.464939,37.91,38.61,76.52,3.93,17.36,1.11,572.235,316.125


In [28]:
# 模型資料 要記得拔除焦870度左右的溫度 和 常溫的資料刪除
print(df.isnull().sum().sum())
def remove_some_data(df):
    for c in tqdm(df.columns[df.columns.str.contains('COT')]):
        lb = df[c].median() - 1.5*df[c].std()
        #ub = df[c].median() + 1.5*df[c].std()
        ub = 870
        df.loc[(df[c]<lb)|(df[c]>ub),c] = None
    return df
df = remove_some_data(df)
print(df.isnull().sum().sum())

0



140977


# 欄位定義

In [29]:
action = df.columns[df.columns.str.contains('COT')].tolist()
value = ['乙烯產量','丙烯產量']
state = set(df.columns) - set(action) - set(value)
state = list(state)
action,value,state

(['H-110 COT',
  'H-120 COT',
  'H-130 COT',
  'H-140 COT',
  'H-150 COT',
  'H-160 COT',
  'H-170 COT',
  'H-180 COT',
  'H-190 COT'],
 ['乙烯產量', '丙烯產量'],
 ['Aromatics', 'O', 'i-P值', 'N', 'n-P值', 'n+i  P值'])

# 資料轉換

In [30]:
mm_state = MinMaxScaler()
mm_action = MinMaxScaler()
mm_value = MinMaxScaler()

df[state] = mm_state.fit_transform(df[state])
df[action] = mm_action.fit_transform(df[action])
df[value] = mm_value.fit_transform(df[value])

# 建立資料集 for 建模使用

In [31]:
def build_dataset(df,n=144): #一天24小時 操作為10分鐘一筆 一天1440分鐘 1440/10 =  144
    data = {
        'state':[],
        'action':[],
        'value':[],
        'idx':[],
        }
    
    for i in tqdm(range(0,len(df)-n,n)):
        curr = df.iloc[i:i+n] #一小段資料
        if np.isnan(curr.values).sum() == 0: #確保這段資料無缺失值
            data['state'].append(curr[state].values) # 這段時間所有狀態
            data['action'].append(curr[action].values) # 這段時間所有操作
            data['value'].append(curr[value].values[-1]) # 這段時間最後一個時間點的產量
            data['idx'].append(curr.index[-1]) #這段時間最後 出料對應的時間點
    
    # reshape
    data['state'] = np.array(data['state'])
    data['action'] = np.array(data['action'])
    data['value'] = np.array(data['value'])
    
    # store mm_scaler
    data['mm_state'] = mm_state
    data['mm_action'] = mm_action
    data['mm_value'] = mm_value
    
    # store col_name
    data['state_col'] = state
    data['action_col'] = action
    data['value_col'] = value
    
    return data

In [32]:
data = build_dataset(df.iloc[1:,:])

In [33]:
data['action'].shape,data['state'].shape,data['value'].shape

((399, 144, 9), (399, 144, 6), (399, 2))

In [34]:
np.isnan(data['action']).sum(),np.isnan(data['state']).sum(),np.isnan(data['value']).sum()

(0, 0, 0)

In [35]:
# 保存
import joblib
print(len(data['idx']))
joblib.dump(data,'../data/dataset.pkl')

399


['../data/dataset.pkl']